In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from IPython.core.display import display, HTML
#set display options for pandas dataframes
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
display(HTML("<style>.container { width:100% !important; }</style>"))
local_path = "/Users/roellk/HMDA/gleif/gleif_parsing/"

In [ ]:
#load GLEIF LEI list
gleif_df = pd.read_csv(local_path+"data/lei_info.txt", sep="|", dtype="object")
fill_cols = ["legal_city", "legal_name", "hq_city", "hq_country"]
gleif_df.fillna("-1", inplace=True)
gleif_df = gleif_df[gleif_df.legal_country=="US"] #filter for only US companies to speed matching
#split legal region to capture only 2 digit state code EG: MT
gleif_df['state'] = gleif_df.legal_region.apply(lambda x: x[-2:] if x != "-1" else "")
gleif_df[gleif_df.state!=""].state.head() #check output

In [ ]:
#load NIC IDs (this is panel with addresses pulled from NIC)
ts_data = pd.read_csv(local_path+"data/ts_addresses.csv", sep=",", dtype="object")
ts_data.columns = ["institution_id", "agency", "rid", "name", "state", "city", "ts_address", "ts_city", "ts_state"]
ts_data.fillna("-1", inplace=True)
print(len(ts_data))
ts_data.dtypes


In [ ]:
# create column with name, address, city, state to match to gleif
ts_data['match_string'] = ts_data.apply(lambda x: str(x.name) + " " + str(x.ts_city) + " " + str(x.ts_state) + " " + str(x.ts_address), axis=1)
ts_data.head()

In [ ]:
#gleif: legal_name, legal_city, legal_region[-2], legal_street1
gleif_df['match_string_gleif'] = gleif_df.apply(lambda x: x.legal_name + " " + x.legal_city + " " +
                                               x.state + " " + x.legal_street1, axis=1)



In [ ]:
count = 0
ts_data["match_tup"] = ""
for index, row in ts_data[ts_data.match_string!="-1 -1 -1 -1"].iterrows():
    print(index, row.match_string)
    #match_output = row.apply(lambda x: process.extractOne(x.match_string, gleif_df.match_string_gleif))
    match_output = process.extractOne(ts_data.at[index, "match_string"], gleif_df.match_string_gleif)
    #print(ts_data.at[index, "match_string"])
    ts_data.at[index, "match_tup"] = match_output
    
    if count % 10 == 0 and count >=10:
        ts_data.to_csv("../output/ts_results_"+str(count)+".txt", sep="|", index=False)
        print("saving {count}".format(count=count))

In [ ]:
ts_data.to_csv("../output/ts_match.txt", sep="|", index=False)

In [ ]:
#split ts_data match tuple
#get LEI by index
#sort by match score, 95 is approximate low end
